In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!pip install duckdb pandas pyarrow

In [ ]:
import duckdb
import os
import pandas as pd
from pathlib import Path

In [ ]:
# Define the path to your Parquet files
parquet_path = '/content/drive/MyDrive/BigDataProject/final_10M_parquet'

# Check if the directory exists
if not os.path.exists(parquet_path):
    print(f"❌ Directory not found: {parquet_path}")
    print("\nPlease check:")
    print(f"1. Is your Parquet data in: {parquet_path}?")
    print("2. If the path is different, update the 'parquet_path' variable above")

    # List what's in the BigDataProject directory
    parent_dir = '/content/drive/MyDrive/BigDataProject'
    if os.path.exists(parent_dir):
        print(f"\n📁 Contents of {parent_dir}:")
        for item in os.listdir(parent_dir):
            print(f"  - {item}")
else:
    print(f"✅ Directory found: {parquet_path}")

    # Count and list Parquet files
    parquet_files = list(Path(parquet_path).glob('*.parquet'))
    if len(parquet_files) == 0:
        parquet_files = list(Path(parquet_path).glob('*.parq'))

    print(f"📊 Found {len(parquet_files)} Parquet file(s)")

    if parquet_files:
        print("\n📄 File list:")
        for i, file in enumerate(parquet_files[:5]):  # Show first 5 files
            print(f"  {i+1}. {file.name} ({file.stat().st_size / (1024*1024):.2f} MB)")
        if len(parquet_files) > 5:
            print(f"  ... and {len(parquet_files) - 5} more files")

✅ Directory found: /content/drive/MyDrive/BigDataProject/final_10M_parquet
📊 Found 200 Parquet file(s)

📄 File list:
  1. part-00000-000f2832-3539-4d78-8af1-8b82adf33860-c000.snappy.parquet (8.51 MB)
  2. part-00001-000f2832-3539-4d78-8af1-8b82adf33860-c000.snappy.parquet (8.50 MB)
  3. part-00002-000f2832-3539-4d78-8af1-8b82adf33860-c000.snappy.parquet (8.51 MB)
  4. part-00003-000f2832-3539-4d78-8af1-8b82adf33860-c000.snappy.parquet (8.50 MB)
  5. part-00004-000f2832-3539-4d78-8af1-8b82adf33860-c000.snappy.parquet (8.50 MB)
  ... and 195 more files


In [ ]:
# Define database path (store it in Google Drive for persistence)
db_path = '/content/drive/MyDrive/BigDataProject/bigdata.db'

# Connect to DuckDB
conn = duckdb.connect(db_path)
print(f"✅ Connected to DuckDB database: {db_path}")

✅ Connected to DuckDB database: /content/drive/MyDrive/BigDataProject/bigdata.db


In [ ]:
# First, let's see what's in the Parquet file
if os.path.exists(parquet_path):
    # Check if it's a single file or directory
    if os.path.isfile(parquet_path):
        # Single Parquet file
        query = f"SELECT * FROM read_parquet('{parquet_path}') LIMIT 5"
    else:
        # Directory with multiple Parquet files
        parquet_pattern = f"{parquet_path}/*.parquet"
        query = f"SELECT * FROM read_parquet('{parquet_pattern}') LIMIT 5"

    try:
        # Preview the data
        print("🔍 Preview of Parquet data (first 5 rows):")
        preview_df = conn.execute(query).df()
        print(f"📊 Shape: {preview_df.shape}")
        print("\n📋 Columns:")
        for col in preview_df.columns:
            print(f"  - {col}")
        print("\n📝 First 5 rows:")
        print(preview_df)

        # Get total row count
        if os.path.isfile(parquet_path):
            count_query = f"SELECT COUNT(*) as total_rows FROM read_parquet('{parquet_path}')"
        else:
            count_query = f"SELECT COUNT(*) as total_rows FROM read_parquet('{parquet_pattern}')"

        total_rows = conn.execute(count_query).fetchone()[0]
        print(f"\n🧮 Total rows in Parquet: {total_rows:,}")

    except Exception as e:
        print(f"❌ Error reading Parquet: {e}")
        print("\nTrying alternative Parquet extension...")
        # Try with .parq extension
        parquet_pattern = f"{parquet_path}/*.parq"
        try:
            query = f"SELECT * FROM read_parquet('{parquet_pattern}') LIMIT 5"
            preview_df = conn.execute(query).df()
            print(f"📊 Shape: {preview_df.shape}")
            print(preview_df)
        except:
            print("Could not read Parquet files. Please check the file format.")

🔍 Preview of Parquet data (first 5 rows):
📊 Shape: (5, 63)

📋 Columns:
  - Type
  - Days for shipping (real)
  - Days for shipment (scheduled)
  - Benefit per order
  - Sales per customer
  - Delivery Status
  - Late_delivery_risk
  - Category Id
  - Category Name
  - Customer City
  - Customer Fname
  - Customer Id
  - Customer Lname
  - Customer Segment
  - Customer Zipcode
  - Department Id
  - Department Name
  - Latitude
  - Longitude
  - Market
  - Order City
  - Order Country
  - Order Customer Id
  - order date (DateOrders)
  - Order Id
  - Order Item Cardprod Id
  - Order Item Discount
  - Order Item Discount Rate
  - Order Item Id
  - Order Item Product Price
  - Order Item Profit Ratio
  - Order Item Quantity
  - Sales
  - Order Item Total
  - Order Profit Per Order
  - Order Region
  - Order Status
  - Product Card Id
  - Product Category Id
  - Product Name
  - Product Price
  - shipping date (DateOrders)
  - Shipping Mode
  - order_date_timestamp
  - order_date_date
  - o

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


🧮 Total rows in Parquet: 10,000,000


In [ ]:
# Create a table in DuckDB from the Parquet files
table_name = "bigdata_table"

print(f"🔄 Creating table '{table_name}' from Parquet files...")

if os.path.exists(parquet_path):
    try:
        # Drop table if it exists
        conn.execute(f"DROP TABLE IF EXISTS {table_name}")

        # Create table from Parquet
        if os.path.isfile(parquet_path):
            # Single file
            create_query = f"""
            CREATE TABLE {table_name} AS
            SELECT * FROM read_parquet('{parquet_path}')
            """
        else:
            # Multiple files (using glob pattern)
            create_query = f"""
            CREATE TABLE {table_name} AS
            SELECT * FROM read_parquet('{parquet_path}/*.parquet')
            """

        conn.execute(create_query)

        # Verify table creation
        row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
        print(f"✅ Successfully created table '{table_name}' with {row_count:,} rows")

        # Show table schema
        print("\n📋 Table schema:")
        schema = conn.execute(f"DESCRIBE {table_name}").df()
        print(schema)

    except Exception as e:
        print(f"❌ Error creating table: {e}")
        print("\nTrying with .parq extension...")
        try:
            create_query = f"""
            CREATE TABLE {table_name} AS
            SELECT * FROM read_parquet('{parquet_path}/*.parq')
            """
            conn.execute(create_query)
            row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
            print(f"✅ Successfully created table '{table_name}' with {row_count:,} rows")
        except Exception as e2:
            print(f"❌ Failed: {e2}")



🔄 Creating table 'bigdata_table' from Parquet files...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Successfully created table 'bigdata_table' with 10,000,000 rows

📋 Table schema:
                      column_name column_type null   key default extra
0                            Type     VARCHAR  YES  None    None  None
1        Days for shipping (real)     INTEGER  YES  None    None  None
2   Days for shipment (scheduled)     INTEGER  YES  None    None  None
3               Benefit per order      DOUBLE  YES  None    None  None
4              Sales per customer      DOUBLE  YES  None    None  None
..                            ...         ...  ...   ...     ...   ...
58              cleaned_timestamp   TIMESTAMP  YES  None    None  None
59                   data_version     VARCHAR  YES  None    None  None
60                  global_row_id     INTEGER  YES  None    None  None
61                   batch_number     INTEGER  YES  None    None  None
62           generation_timestamp   TIMESTAMP  YES  None    None  None

[63 rows x 6 columns]


In [ ]:
# Example 1: Basic query
print("📊 Example Query 1: First 10 rows")
query1 = f"SELECT * FROM {table_name} LIMIT 10"
result1 = conn.execute(query1).df()
print(result1)

📊 Example Query 1: First 10 rows
       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0  TRANSFER                         2                              4   
1      CASH                         6                              4   
2      CASH                         3                              4   
3     DEBIT                         0                              0   
4     DEBIT                         4                              2   
5     DEBIT                         3                              4   
6      CASH                         5                              4   
7  TRANSFER                         2                              1   
8   PAYMENT                         5                              4   
9     DEBIT                         4                              4   

   Benefit per order  Sales per customer   Delivery Status  \
0         105.618510          216.787260  Advance shipping   
1           2.098613           18.424766     Late 

In [ ]:
# Example 2: Count by a column (if you have categorical data)
print("\n📊 Example Query 2: Summary statistics")

# Get column names to suggest possible categorical columns
columns = conn.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name='{table_name}'").df()
print(f"Available columns: {list(columns['column_name'])}")

# Try to find a categorical column for demonstration
try:
    # Look for columns that might be categorical (string type)
    for col in columns['column_name']:
        col_type = conn.execute(f"""
            SELECT data_type
            FROM information_schema.columns
            WHERE table_name='{table_name}' AND column_name='{col}'
        """).fetchone()[0]

        if 'char' in col_type.lower() or 'text' in col_type.lower():
            query2 = f"""
            SELECT {col}, COUNT(*) as count
            FROM {table_name}
            GROUP BY {col}
            ORDER BY count DESC
            LIMIT 5
            """
            print(f"\nTop 5 values in '{col}' column:")
            result2 = conn.execute(query2).df()
            print(result2)
            break
except:
    # If no categorical column, show numeric stats
    for col in columns['column_name']:
        col_type = conn.execute(f"""
            SELECT data_type
            FROM information_schema.columns
            WHERE table_name='{table_name}' AND column_name='{col}'
        """).fetchone()[0]

        if 'int' in col_type.lower() or 'float' in col_type.lower() or 'double' in col_type.lower():
            query2 = f"""
            SELECT
                MIN({col}) as min_value,
                MAX({col}) as max_value,
                AVG({col}) as avg_value,
                COUNT({col}) as non_null_count
            FROM {table_name}
            """
            print(f"\nStatistics for '{col}' column:")
            result2 = conn.execute(query2).df()
            print(result2)
            break


📊 Example Query 2: Summary statistics
Available columns: ['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Delivery Status', 'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Customer Segment', 'Customer Zipcode', 'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market', 'Order City', 'Order Country', 'Order Customer Id', 'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Profit Per Order', 'Order Region', 'Order Status', 'Product Card Id', 'Product Category Id', 'Product Name', 'Product Price', 'shipping date (DateOrders)', 'Shipping Mode', 'order_date_timestamp', 'order_date_date', 'order_date_year', 'order_date_month', 'order_date_day', 'orde

In [ ]:
# Check if the database file exists and show its details
print("📁 Database File Location:")
print(f"Path: {db_path}")

# Check if file exists
if os.path.exists(db_path):
    # Get file size
    file_size_bytes = os.path.getsize(db_path)
    file_size_mb = file_size_bytes / (1024 * 1024)

    # Get file creation/modification time
    from datetime import datetime
    mtime = os.path.getmtime(db_path)
    mod_time = datetime.fromtimestamp(mtime).strftime('%Y-%m-%d %H:%M:%S')

    print(f"✅ Database file exists!")
    print(f"📏 File size: {file_size_mb:.2f} MB ({file_size_bytes:,} bytes)")
    print(f"🕒 Last modified: {mod_time}")

    # Also list the BigDataProject directory to show the file
    print(f"\n📂 Contents of BigDataProject directory:")
    project_dir = '/content/drive/MyDrive/BigDataProject'
    if os.path.exists(project_dir):
        items = os.listdir(project_dir)
        for item in items:
            item_path = os.path.join(project_dir, item)
            if os.path.isdir(item_path):
                print(f"  📁 {item}/")
            else:
                # Show size for files
                size_bytes = os.path.getsize(item_path)
                size_mb = size_bytes / (1024 * 1024)
                if size_mb > 1:
                    print(f"  📄 {item} ({size_mb:.2f} MB)")
                else:
                    size_kb = size_bytes / 1024
                    print(f"  📄 {item} ({size_kb:.2f} KB)")
    else:
        print(f"❌ Directory not found: {project_dir}")
else:
    print(f"❌ Database file NOT found at: {db_path}")
    print("\n⚠️  The database was created in memory, not saved to disk!")
    print("   To save it, run: conn.close() to flush changes to disk.")

# Important: Close the connection to ensure everything is saved to disk
conn.close()
print(f"\n🔒 Connection closed. Database saved to: {db_path}")

📁 Database File Location:
Path: /content/drive/MyDrive/BigDataProject/bigdata.db
✅ Database file exists!
📏 File size: 1519.76 MB (1,593,585,664 bytes)
🕒 Last modified: 2025-12-30 10:10:31

📂 Contents of BigDataProject directory:
  📁 cleaned_data.csv/
  📁 cleaned_data.parquet/
  📁 augmented/
  📁 final_10M_parquet/
  📁 cleaned_data_with_features.parquet/
  📄 cleaned_data_with_features.csv (128.70 MB)
  📄 bigdata.db (1519.76 MB)

🔒 Connection closed. Database saved to: /content/drive/MyDrive/BigDataProject/bigdata.db
